<a href="https://colab.research.google.com/github/aadityane93/Machine_Learning_Notebooks/blob/main/Sorting_line_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tsfresh
# getml --upgrade
from google.colab import files
import numpy as np
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 54.1 MB/s eta 0:00:00


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1BZJfMkmr5uyyL1iZGPQen-B3VTYrJjD0' #100 files
# folder_id = '1bxp8LJq3826UeOR44M9ZiDFX61Dp-xSf' #all files
# folder_id = '1Goeo7w4XQ_BRZVN8zRX3t11Ollg811VB' #10 files
folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        new_file_name = f"{folder_title}-{index}"
        print(f"Downloading {file_title} as {new_file_name}")
        file.GetContentFile(new_file_name)

Processing folder: niO
Processing folder: biO
Processing folder: riO
Processing folder: wiO


In [3]:
index = 100

#test train percentage
test_train_split_percentage = 20    # __% percent of data files will be used to train the model

print ("total no. of files for each color  - ", index)
print ("total no. of files  - ", index*4)
print (" ")

#Index to seperarate the read data into train and test dataframes
index_train = (test_train_split_percentage/100)* index
index_train = int(index_train)
index_test = 100 - index_train

print (f"Percent of training data - {test_train_split_percentage}%")
print (f"No. of files for training - {index_train}")
print (" ")
print (f"Percent of testing data - {100 - test_train_split_percentage}%")
print (f"No. of files for testing - {index_test}")

total no. of files for each color  -  100
total no. of files  -  400
 
Percent of training data - 20%
No. of files for training - 20
 
Percent of testing data - 80%
No. of files for testing - 80


In [52]:
def read_file(filename):
    return pd.read_csv((filename), sep=',')
########################################################## INDEX ################################################

# Dictonaries to read the files
white_train = {}
white_test = {}
blue_train = {}
blue_test = {}
red_train = {}
red_test = {}
nio_train = {}
nio_test = {}

#Assigning df
white_train_df = pd.DataFrame()
white_test_df = pd.DataFrame()
blue_train_df = pd.DataFrame()
blue_test_df = pd.DataFrame()
red_train_df = pd.DataFrame()
red_test_df = pd.DataFrame()
nio_train_df = pd.DataFrame()
nio_test_df = pd.DataFrame()

#Reading files for training
for j in range (index_train):
  white_train[j] = read_file(f"wiO-{j}")
  blue_train[j] = read_file(f"biO-{j}")
  red_train[j] = read_file(f"riO-{j}")
  nio_train[j] = read_file(f"niO-{j}")
  print(j)

print("\n")
#Reading files for testing
for i in range (index_test):
  white_test[i] = read_file(f"wiO-{i+index_train}")
  blue_test[i] = read_file(f"biO-{i+index_train}")
  red_test[i] = read_file(f"riO-{i+index_train}")
  nio_test[i] = read_file(f"niO-{i+index_train}")
  print (i+index_train)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [53]:
white_train

{0:           Datum                         Uhrzeit  I_In   V_In  P_In(W)  \
 0    2024-05-02  2024-05-02 11:37:37.940396:940  0.25  11.65   2.9125   
 1    2024-05-02  2024-05-02 11:37:37.948287:948  0.27  11.52   3.1104   
 2    2024-05-02  2024-05-02 11:37:37.956565:956  0.31  11.65   3.6115   
 3    2024-05-02  2024-05-02 11:37:37.968670:968  0.31  11.65   3.6115   
 4    2024-05-02  2024-05-02 11:37:37.977001:976  0.37  11.65   4.3105   
 ..          ...                             ...   ...    ...      ...   
 618  2024-05-02  2024-05-02 11:37:44.129232:129  0.39  11.65   4.5435   
 619  2024-05-02  2024-05-02 11:37:44.137453:137  0.46  11.70   5.3820   
 620  2024-05-02  2024-05-02 11:37:44.149598:149  0.50  11.65   5.8250   
 621  2024-05-02  2024-05-02 11:37:44.157892:157  0.43  11.65   5.0095   
 622  2024-05-02  2024-05-02 11:37:44.170060:170  0.44  11.65   5.1260   
 
      I_Out(A)  V_Out(V)  P_Out(W)  Temp(°C)  Energie_In(Wh)  Energie_Out(Wh)  
 0        0.29      8.97   

In [55]:
# Color df for Train Dataset
print("White Train ", len(white_train), " ", "Blue Train ", len(blue_train), " ", "Red Train ", len(red_train), " ", "Nio Train ", len(nio_train))

df_color_train = pd.DataFrame()
for i in range(len(white_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(0)])
for i in range(len(blue_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(1)])
for i in range(len(red_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(2)])
for i in range(len(nio_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(3)])
df_color_train = df_color_train.astype(int)
df_color_train.columns = ['color']


# Color df for Test Dataset
print("White Test ", len(white_test), " ", "Blue Test ", len(blue_test), " ", "Red Test ", len(red_test), " ", "Nio Test ", len(nio_test))
df_color_test = pd.DataFrame()
for i in range(len(white_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(0)])
for i in range(len(blue_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(1)])
for i in range(len(red_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(2)])
for i in range(len(nio_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(3)])
df_color_test = df_color_test.astype(int)
df_color_test.columns = ['color']

White Train  20   Blue Train  20   Red Train  20   Nio Train  20
White Test  80   Blue Test  80   Red Test  80   Nio Test  80


In [56]:
print(df_color_train.shape)
print(df_color_test.shape)

(80, 1)
(320, 1)


In [57]:
# Merging df of different Colors into a single df for Train Dataset

q= 0
r= index_train + index_test
s = index_train + index_test + r
t = index_train + index_test + s


for j in range(index_train):
  print (' q - ',q,' r - ',r,' s - ',s,' t - ',t)
  white_train[j] = pd.DataFrame(white_train[j])
  white_train[j]['fileno'] = q
  white_train_df = pd.concat([white_train_df, white_train[j]])

  blue_train[j] = pd.DataFrame(blue_train[j])
  blue_train[j]['fileno'] = r
  blue_train_df = pd.concat([blue_train_df, blue_train[j]])

  red_train[j] = pd.DataFrame(red_train[j])
  red_train[j]['fileno'] = s
  red_train_df = pd.concat([red_train_df, red_train[j]])

  nio_train[j] = pd.DataFrame(nio_train[j])
  nio_train[j]['fileno'] = t
  nio_train_df = pd.concat([nio_train_df, nio_train[j]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1



# Merging df of different Colors into a single df for Test Dataset

q= index_train
r= r
s = s
t = t

print (" \n")

for j in range(index_test):
  print (' q - ',q,' r - ',r,' s - ',s,' t - ',t)
  white_test[j] = pd.DataFrame(white_test[j])
  white_test[j]['fileno'] = q
  white_test_df = pd.concat([white_test_df, white_test[j]])

  blue_test[j] = pd.DataFrame(blue_test[j])
  blue_test[j]['fileno'] = r
  blue_test_df = pd.concat([blue_test_df, blue_test[j]])

  red_test[j] = pd.DataFrame(red_test[j])
  red_test[j]['fileno'] = s
  red_test_df = pd.concat([red_test_df, red_test[j]])

  nio_test[j] = pd.DataFrame(nio_test[j])
  nio_test[j]['fileno'] = t
  nio_test_df = pd.concat([nio_test_df, nio_test[j]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1

 q -  0  r -  100  s -  200  t -  300
 q -  1  r -  101  s -  201  t -  301
 q -  2  r -  102  s -  202  t -  302
 q -  3  r -  103  s -  203  t -  303
 q -  4  r -  104  s -  204  t -  304
 q -  5  r -  105  s -  205  t -  305
 q -  6  r -  106  s -  206  t -  306
 q -  7  r -  107  s -  207  t -  307
 q -  8  r -  108  s -  208  t -  308
 q -  9  r -  109  s -  209  t -  309
 q -  10  r -  110  s -  210  t -  310
 q -  11  r -  111  s -  211  t -  311
 q -  12  r -  112  s -  212  t -  312
 q -  13  r -  113  s -  213  t -  313
 q -  14  r -  114  s -  214  t -  314
 q -  15  r -  115  s -  215  t -  315
 q -  16  r -  116  s -  216  t -  316
 q -  17  r -  117  s -  217  t -  317
 q -  18  r -  118  s -  218  t -  318
 q -  19  r -  119  s -  219  t -  319
 

 q -  20  r -  120  s -  220  t -  320
 q -  21  r -  121  s -  221  t -  321
 q -  22  r -  122  s -  222  t -  322
 q -  23  r -  123  s -  223  t -  323
 q -  24  r -  124  s -  224  t -  324
 q -  25  r -  125  s -  225  t 

In [8]:
# for i in range (len(white_df)):
#   white_df['fileno'] = i
# for j in range (i, i + len(blue_df)):
#   blue_df['fileno'].append(j)
# for k in range (j,j+len(red_df)):
#   red_df['fileno'].append(j)
# for l in range (k, k+len(nio_df)):
#   nio_df['fileno'].append(j)

In [58]:
#Assigning color to the the dataframe
white_train_df ['color'] = 0
blue_train_df ['color'] = 1
red_train_df ['color'] = 2
nio_train_df ['color'] = 3

white_test_df ['color'] = 0
blue_test_df ['color'] = 1
red_test_df ['color'] = 2
nio_test_df ['color'] = 3

In [59]:
df_train = pd.concat([white_train_df, blue_train_df, red_train_df, nio_train_df])
df_test = pd.concat([white_test_df, blue_test_df, red_test_df, nio_test_df])


df_train = df_train.drop(columns=['Datum'])
df_test = df_test.drop(columns=['Datum'])

df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

#For tsfresh
x_train_df = df_train.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)','color'])
x_test_df = df_test.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)','color'])

# y = df['color']
# j=0
# y_aggregated = pd.DataFrame()
# for i in range(len(df['fileno'])):
#   if df['fileno'].values[i] ==



  # if df['color'].values[i] == j:
  #   y_aggregated = pd.concat([y_aggregated, pd.Series(j)])
  # elif df['color'].values[i] != j:
  #   j = j + 1
  #   y_aggregated = pd.concat([y_aggregated, pd.Series(j)])
  # else:
  #   print('error')


In [60]:
print(df_train['fileno'])

0          0
1          0
2          0
3          0
4          0
        ... 
59893    319
59894    319
59895    319
59896    319
59897    319
Name: fileno, Length: 59898, dtype: int64


In [61]:
features_train =  extract_features(
    x_train_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

Feature Extraction: 100%|██████████| 400/400 [06:09<00:00,  1.08it/s]


In [62]:
# feature_file = f'features-{index*4}.csv'
# features.to_csv(feature_file, sep=',')
# files.download(feature_file)

In [63]:
features_train.head()

,I_In__variance_larger_than_standard_deviation,I_In__has_duplicate_max,I_In__has_duplicate_min,I_In__has_duplicate,I_In__sum_values,I_In__abs_energy,I_In__mean_abs_change,I_In__mean_change,I_In__mean_second_derivative_central,I_In__median,...,Temp(°C)__fourier_entropy__bins_5,Temp(°C)__fourier_entropy__bins_10,Temp(°C)__fourier_entropy__bins_100,Temp(°C)__permutation_entropy__dimension_3__tau_1,Temp(°C)__permutation_entropy__dimension_4__tau_1,Temp(°C)__permutation_entropy__dimension_5__tau_1,Temp(°C)__permutation_entropy__dimension_6__tau_1,Temp(°C)__permutation_entropy__dimension_7__tau_1,Temp(°C)__query_similarity_count__query_None__threshold_0.0,Temp(°C)__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,189.99,60.2121,0.042299,0.000305,-8.051530e-06,0.29,...,1.204209,1.795131,3.805391,1.318114,2.152424,2.987462,3.746941,4.391745,0.0,37.88
1,0.0,0.0,1.0,1.0,185.58,59.6738,0.042040,0.000216,0.000000e+00,0.29,...,1.436031,2.089244,4.045644,1.410305,2.314589,3.235639,4.075079,4.799483,0.0,37.63
2,0.0,0.0,1.0,1.0,184.09,59.0125,0.043344,0.000250,-4.625929e-20,0.30,...,1.139872,1.786103,3.792321,1.462869,2.450789,3.414714,4.310813,5.059538,0.0,37.63
3,0.0,0.0,0.0,1.0,187.40,60.1416,0.042311,0.000246,-1.642036e-05,0.29,...,1.219551,1.839760,3.856714,1.485377,2.524549,3.559655,4.512030,5.219138,0.0,37.63
4,0.0,0.0,1.0,1.0,183.78,59.4834,0.041930,0.000285,1.680672e-05,0.29,...,1.211701,1.898553,3.854095,1.467557,2.479623,3.458075,4.381721,5.129161,0.0,37.63


In [64]:
print(features_train.shape)
y_train_df= df_color_train['color']
y_train_df = y_train_df.reset_index(drop=True)
print(y_train_df.shape)

(80, 3915)
(80,)


In [65]:
y_train_df

,color
0,0
1,0
2,0
3,0
4,0
...,...
75,3
76,3
77,3
78,3


In [66]:
features_train = features_train.reset_index(drop=True)
features_train

,I_In__variance_larger_than_standard_deviation,I_In__has_duplicate_max,I_In__has_duplicate_min,I_In__has_duplicate,I_In__sum_values,I_In__abs_energy,I_In__mean_abs_change,I_In__mean_change,I_In__mean_second_derivative_central,I_In__median,...,Temp(°C)__fourier_entropy__bins_5,Temp(°C)__fourier_entropy__bins_10,Temp(°C)__fourier_entropy__bins_100,Temp(°C)__permutation_entropy__dimension_3__tau_1,Temp(°C)__permutation_entropy__dimension_4__tau_1,Temp(°C)__permutation_entropy__dimension_5__tau_1,Temp(°C)__permutation_entropy__dimension_6__tau_1,Temp(°C)__permutation_entropy__dimension_7__tau_1,Temp(°C)__query_similarity_count__query_None__threshold_0.0,Temp(°C)__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,189.99,60.2121,0.042299,0.000305,-8.051530e-06,0.29,...,1.204209,1.795131,3.805391,1.318114,2.152424,2.987462,3.746941,4.391745,0.0,37.880000
1,0.0,0.0,1.0,1.0,185.58,59.6738,0.042040,0.000216,0.000000e+00,0.29,...,1.436031,2.089244,4.045644,1.410305,2.314589,3.235639,4.075079,4.799483,0.0,37.630000
2,0.0,0.0,1.0,1.0,184.09,59.0125,0.043344,0.000250,-4.625929e-20,0.30,...,1.139872,1.786103,3.792321,1.462869,2.450789,3.414714,4.310813,5.059538,0.0,37.630000
3,0.0,0.0,0.0,1.0,187.40,60.1416,0.042311,0.000246,-1.642036e-05,0.29,...,1.219551,1.839760,3.856714,1.485377,2.524549,3.559655,4.512030,5.219138,0.0,37.630000
4,0.0,0.0,1.0,1.0,183.78,59.4834,0.041930,0.000285,1.680672e-05,0.29,...,1.211701,1.898553,3.854095,1.467557,2.479623,3.458075,4.381721,5.129161,0.0,37.630000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.0,0.0,1.0,1.0,271.25,85.5081,0.044117,0.000022,-1.520270e-04,0.31,...,1.498794,2.141156,4.082476,1.396455,2.306869,3.210353,4.052507,4.811570,0.0,38.237143
76,0.0,0.0,1.0,1.0,266.49,84.6543,0.045155,-0.000046,-1.152074e-05,0.31,...,1.216399,1.848685,3.889092,1.331199,2.195858,3.079873,3.917019,4.664015,0.0,38.380000
77,0.0,0.0,1.0,1.0,262.53,83.8761,0.047057,0.000188,-2.934272e-05,0.31,...,1.233626,1.904185,3.886856,1.334239,2.235023,3.131234,3.966831,4.651828,0.0,38.380000
78,0.0,0.0,0.0,1.0,274.21,87.3141,0.045112,0.000000,-2.234637e-05,0.31,...,1.352589,1.980862,3.999675,1.373644,2.275224,3.184214,3.992090,4.690126,0.0,38.380000


In [67]:
selected_features = select_features(features_train, y_train_df)

In [68]:
selected_features.head()

,I_In__number_peaks__n_1,I_Out(A)__count_above_mean,I_In__count_above_mean,I_Out(A)__range_count__max_1__min_-1,I_In__range_count__max_1__min_-1,Temp(°C)__length,I_Out(A)__length,I_In__length,V_In__length,Temp(°C)__range_count__max_1000000000000.0__min_0,...,Temp(°C)__large_standard_deviation__r_0.25,Temp(°C)__large_standard_deviation__r_0.2,"I_In__linear_trend__attr_""pvalue""","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.2","Temp(°C)__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.2","Temp(°C)__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.2","I_Out(A)__fft_coefficient__attr_""abs""__coeff_18","Temp(°C)__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.4","Temp(°C)__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.4",Temp(°C)__ratio_beyond_r_sigma__r_2.5
0,148.0,201.0,303.0,622.0,623.0,623.0,623.0,623.0,623.0,623.0,...,0.0,0.0,1.188615e-11,0.105332,0.015238,0.026331,3.116657,0.002440,0.002348,0.022472
1,135.0,214.0,301.0,602.0,603.0,604.0,604.0,604.0,604.0,604.0,...,1.0,1.0,1.801481e-12,0.000000,0.000000,0.000000,3.664878,0.028756,0.015526,0.043046
2,136.0,207.0,301.0,601.0,601.0,602.0,602.0,602.0,602.0,602.0,...,1.0,1.0,3.289738e-14,0.000000,0.000000,0.000000,2.346824,0.030682,0.015620,0.000000
3,146.0,189.0,304.0,610.0,610.0,611.0,611.0,611.0,611.0,611.0,...,1.0,1.0,2.313044e-14,0.000000,0.000000,0.000000,3.026270,0.030092,0.015604,0.000000
4,122.0,200.0,289.0,596.0,596.0,597.0,597.0,597.0,597.0,597.0,...,1.0,1.0,1.093517e-10,0.000000,0.000000,0.000000,2.330833,0.030662,0.015619,0.000000


In [69]:
x_train,x_test,y_train,y_test = train_test_split(selected_features,y_train_df, test_size=0.5, random_state=42, stratify=y_train_df)

In [70]:
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00        10

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

[[10  0  0  0]
 [ 0 10  0  0]
 [ 0  0 10  0]
 [ 0  0  0 10]]


In [71]:
def xgb_(X, y, num_folds=5):

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=24)

    f1_scores = []
    accuracies = []
    precisions = []

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold_idx + 1}:")

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Initialize and train the XGBoost model with regularization
        xgb1 = xgb.XGBClassifier(
            objective="multi:softmax",
            random_state=42,
            alpha=0.1,
            reg_lambda=0.1,
            max_depth=4,
            min_child_weight=10,
            subsample=0.8,
            colsample_bytree=0.8
        )

        # Fit the model
        xgb1.fit(X_train.to_numpy(), y_train.to_numpy().ravel())

        # Make predictions
        y_pred = xgb1.predict(X_test.to_numpy())

        # Ensure y_test is 1D for comparison
        y_test = y_test.to_numpy().ravel().astype(int)
        y_pred = y_pred.astype(int)

        # Calculate and store F1-score for this fold
        f1 = f1_score(y_test, y_pred, average='weighted')
        f1_scores.append(f1)

        # Calculate and store accuracy and precision for this fold
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        precision = precision_score(y_test, y_pred, average='weighted')
        precisions.append(precision)

        print(classification_report(y_test, y_pred))
        print(f"Fold {fold_idx + 1} F1-score: {f1:.4f}")
        print(f"Fold {fold_idx + 1} Accuracy: {accuracy:.4f}")
        print(f"Fold {fold_idx + 1} Precision: {precision:.4f}")

        # Calculate and print confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print("Confusion Matrix:")
        print(cm)

        print("-" * 30)

    # Calculate and print average metrics across all folds
    avg_f1 = np.mean(f1_scores)
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)

    print(f"Average F1-score across {num_folds} folds: {avg_f1:.4f}")
    print(f"Average Accuracy across {num_folds} folds: {avg_accuracy:.4f}")
    print(f"Average Precision across {num_folds} folds: {avg_precision:.4f}")

    # Final model: Train on the entire dataset
    print("Training final model on the entire dataset...")
    X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X, y, test_size=0.2, random_state=42)

    # Final model with regularization
    final_model = xgb.XGBClassifier(
        objective="multi:softmax",
        random_state=42,
        alpha=0.1,
        reg_lambda=0.1,
        max_depth=5,
        min_child_weight=10,
        subsample=0.8,
        colsample_bytree=0.8
    )

    # Train on the entire dataset
    final_model.fit(X_train_final.to_numpy(), y_train_final.to_numpy().ravel())

    # Evaluate the final model on the test set
    y_pred_final = final_model.predict(X_test_final.to_numpy())

    # Calculate accuracy, precision, confusion matrix for final model
    final_accuracy = accuracy_score(y_test_final, y_pred_final)
    final_precision = precision_score(y_test_final, y_pred_final, average='weighted')
    final_cm = confusion_matrix(y_test_final, y_pred_final)

    print(f"Final Model Accuracy: {final_accuracy:.4f}")
    print(f"Final Model Precision: {final_precision:.4f}")
    print("Final Model Confusion Matrix:")
    print(final_cm)

    # Save the final model
    # with open('XGB_final_model.pkl', 'wb') as f:
    #     pickle.dump(final_model, f)
    # files.download('XGB_final_model.pkl')



In [72]:
xgb_ (selected_features, y_train_df)

Fold 1:
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.83      0.91         6
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         4

    accuracy                           0.94        16
   macro avg       0.94      0.96      0.94        16
weighted avg       0.95      0.94      0.94        16

Fold 1 F1-score: 0.9391
Fold 1 Accuracy: 0.9375
Fold 1 Precision: 0.9531
Confusion Matrix:
[[3 0 0 0]
 [1 5 0 0]
 [0 0 3 0]
 [0 0 0 4]]
------------------------------
Fold 2:
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.50      0.75      0.60         4
           2       1.00      0.43      0.60         7
           3       0.50      1.00      0.67         3

    accuracy                           0.62        16
   macro avg       0.75      0.67      0.63        16
we

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

              precision    recall  f1-score   support

           0       1.00      0.10      0.18        10
           1       0.57      1.00      0.73         4
           2       0.00      0.00      0.00         0
           3       1.00      1.00      1.00         2

    accuracy                           0.44        16
   macro avg       0.64      0.53      0.48        16
weighted avg       0.89      0.44      0.42        16

Fold 5 F1-score: 0.4205
Fold 5 Accuracy: 0.4375
Fold 5 Precision: 0.8929
Confusion Matrix:
[[1 3 6 0]
 [0 4 0 0]
 [0 0 0 0]
 [0 0 0 2]]
------------------------------
Average F1-score across 5 folds: 0.6377
Average Accuracy across 5 folds: 0.6625
Average Precision across 5 folds: 0.7716
Training final model on the entire dataset...
Final Model Accuracy: 0.7500
Final Model Precision: 0.8646
Final Model Confusion Matrix:
[[5 0 0 0]
 [0 2 1 3]
 [0 0 2 0]
 [0 0 0 3]]


In [73]:
features_test = extract_features(
    x_test_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

Feature Extraction: 100%|██████████| 1600/1600 [24:16<00:00,  1.10it/s]


In [74]:
#Using the same columns from the training data
features_test = features_test[selected_features.columns]

In [75]:
print(features_test.shape)
y_test_df= df_color_test['color']
y_test_df = y_test_df.reset_index(drop=True)
print(y_test_df.shape)

(320, 795)
(320,)


In [76]:
# model = RandomForestClassifier()
# model.fit(features_test, y_test)
y_pred = model.predict(features_test)
accuracy = accuracy_score(y_test_df, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test_df, y_pred))
print(confusion_matrix(y_test_df, y_pred))

Accuracy: 0.86875
              precision    recall  f1-score   support

           0       0.99      0.99      0.99        80
           1       0.90      0.56      0.69        80
           2       0.71      0.99      0.82        80
           3       0.96      0.94      0.95        80

    accuracy                           0.87       320
   macro avg       0.89      0.87      0.86       320
weighted avg       0.89      0.87      0.86       320

[[79  0  1  0]
 [ 0 45 32  3]
 [ 1  0 79  0]
 [ 0  5  0 75]]


In [77]:
# xgb_ (features_test, y_test_df)

In [78]:
# Read file into DataFrame

wio15_df = pd.DataFrame(pd.read_csv(('wiO-15'), sep=',', header=0))



wio15_df['fileno'] = 15
wio15_df = wio15_df.drop(columns=['Datum','P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])

extracted_features = extract_features(
    wio15_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)
wio_features = extracted_features[selected_features.columns]
y_pred = model.predict(wio_features)
print(y_pred)
accuracy = accuracy_score(pd.Series([0]), y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(pd.Series([0]), y_pred))
print(confusion_matrix(pd.Series([0]), y_pred))


Feature Extraction: 100%|██████████| 5/5 [00:03<00:00,  1.46it/s]


[0]
Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

[[1]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [79]:
# #For getML
# x_train_df2 = df_train.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])
# x_test_df2 = df_test.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])

# # x_train_df2 = x_train_df.copy(deep=True)

# import getml

# getml.engine.launch()
# getml.set_project("sorting-line")

# # Split at the last colon
# x_train_df2["Uhrzeit"] = (
#     x_train_df2["Uhrzeit"]
#     .str.rsplit(pat=':', n=1)
#     .str[0]
# )

# # Convert to datetime with flexible parsing
# x_train_df2["Uhrzeit"] = pd.to_datetime(
#     x_train_df2["Uhrzeit"],
#     format='mixed',
#     errors='coerce'
# )
# pop = getml.data.DataFrame.from_pandas(x_train_df2, name = "pop")


# pop.set_role("fileno", getml.data.roles.join_key)
# pop.set_role(["I_In", "V_In", "I_Out(A)", "V_Out(V)", "Temp(°C)"], getml.data.roles.numerical)
# pop.set_role("Uhrzeit", getml.data.roles.time_stamp)
# pop.set_role("color", getml.data.roles.target)

In [80]:
# x_train_df2

In [81]:
# pop